In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://myanimelist.net/topanime.php'
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.title

In [ ]:
animes = soup.find('table', class_='top-ranking-table').find_all('tr', class_='ranking-list')

In [ ]:
len(animes)

In [ ]:
def fetchAnimeInfo(url):
    response = requests.get(url)
    store = list()


    # INFO FROM THE RIGHT PANEL OF THE WINDOW
    soup = BeautifulSoup(response.text, 'html.parser').tr.select_one('td.borderClass')

    # extract useful infos
    aInfos = soup.find_all('div', class_='spaceit_pad')

    # tag words
    tagWords = ['Type:', 'Premiered:', 'Producers:', 'Licensors:', 'Studios:', 'Genres:', 'Theme:',
               'Demographic:']

    for ainfo in aInfos:
        try:
            if ainfo.span.text not in tagWords:
                store.append(ainfo.find('span', class_='dark_text').next_sibling.strip())
            else:
                aTags = ainfo.find_all('a')
                tags = list()
                for atag in aTags:
                    tags.append(atag.text)
                store.append(tags)
        except:
            pass


    # INFO FROM THE LEFT PANEL OF THE WINDOW
    soup2 = BeautifulSoup(response.text, 'html.parser')

    # extract useful infos2
    aInfos2 = soup2.find('div', class_='rightside js-scrollfix-bottom-rel').table

    store.append(aInfos2.p.text)
    store.append(aInfos2.find('div', class_='score-label').text)



    return store

# span_element = div_element.find('span', class_='dark-text')
# text = span_element.next_sibling.strip()

In [ ]:
for anime in animes:
    anime_url = anime.find('td', class_='title al va-t word-break').a.get('href')
    mega_anime = list()
    mega_anime.append(fetchAnimeInfo(anime_url))


# create a dictionary
anime_dict = dict()
keys = ['synonyms', 'japanese_title', 'english_title', 'french_title', 'format', 'episodes',
        'status', 'aired', 'season', 'broadcast', 'producers', 'licensors',
        'studios', 'source', 'genres', 'theme', 'demographic', 'duration', 'rating', 'whtevr',
        'rank', 'popularity', 'members', 'favorites', 'synopsis', 'rating_value']


# store anime info in a dictionary
for i in range(26):
    getStored = list()
    for j in range(len(mega_anime)):
        getStored.append(mega_anime[j][i])

anime_dict[keys[i]] = getStored

In [ ]:
fetchAnimeInfo('https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood')

In [ ]:
print(anime_dict)

In [ ]:
print(len(keys))

## SCRAPE TOP ANIMES FROM ZORO

In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def fetchAnimeContent(anime: bs4.element.Tag):
    # forwarding to anime link
    forwardLink = f"https://zoro.to{anime.find('div', class_='film-poster').a['href']}"
    response = requests.get(forwardLink)
    soup = BeautifulSoup(response.text, 'html.parser')
    moreAttr = soup.find('div', class_='ani_detail-stage').find('div', class_='container')

    # extract anime attributes needed
    title = anime.find('div', class_='film-detail').find('h3', class_='film-name').a['title']
    description = moreAttr.find('div', class_='film-description m-hide').div.text.strip()
    genres = [genre.text for genre in moreAttr.find('div', class_='item item-list').find_all('a')]
    poster = anime.find('div', class_='film-poster').find('img')['data-src']

    return title, genres, description, poster

In [ ]:
try:
    # fetch the website url
    url = 'https://zoro.to/most-popular'
    response = requests.get(url)

    # fetch the anime page
    soup = BeautifulSoup(response.text, 'html.parser')
    page = soup.find('div', class_='tab-content')
    animes = page.find_all('div', class_='flw-item')

    # container to store anime attributes
    storeIt = {'title': [], 'genres': [], 'description': [], 'poster': []}

    # start scraping anime
    for i in range(1, 51):
        for anime in animes:
            try:
                # finally start storing the anime attributes
                title, genres, description, poster = fetchAnimeContent(anime)
                storeIt['title'].append(title)
                storeIt['genres'].append(genres)
                storeIt['description'].append(description)
                storeIt['poster'].append(poster)
            except:
                continue

        # change the animes page link
        url = f'https://zoro.to/most-popular?page={i+1}'
        response = requests.get(url)

        # fetch the anime page
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find('div', class_='tab-content')
        animes = page.find_all('div', class_='flw-item')

except Exception as error:
    print('Error Occured:', error)

In [ ]:
data = pd.DataFrame(storeIt)

In [ ]:
data.shape

(1999, 4)

## SCRAPE ANIME MOVIES FROM ZORO

In [ ]:
try:
    # fetch the website url
    url = 'https://zoro.to/movie'
    response = requests.get(url)

    # fetch the anime page
    soup = BeautifulSoup(response.text, 'html.parser')
    page = soup.find('div', class_='tab-content')
    animes = page.find_all('div', class_='flw-item')

    # container to store anime attributes
    moviesContainer = {'title': [], 'genres': [], 'description': [], 'poster': []}

    # start scraping anime
    for i in range(1, 26):
        for anime in animes:
            try:
                # finally start storing the anime attributes
                title, genres, description, poster = fetchAnimeContent(anime)
                moviesContainer['title'].append(title)
                moviesContainer['genres'].append(genres)
                moviesContainer['description'].append(description)
                moviesContainer['poster'].append(poster)
            except:
                continue

        # change the animes page link
        url = f'https://zoro.to/movie?page={i+1}'
        response = requests.get(url)

        # fetch the anime page
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find('div', class_='tab-content')
        animes = page.find_all('div', class_='flw-item')
except Exception as error:
    print('Error Occured:', error)

In [ ]:
movies = pd.DataFrame(moviesContainer)

In [ ]:
movies.shape

(865, 4)

In [ ]:
data.to_csv('/content/drive/MyDrive/csv_files/animes.csv')
movies.to_csv('/content/drive/MyDrive/csv_files/movies.csv')

## SCRAPE MOST FAVORITE ANIMES FROM ZORO

In [ ]:
try:
    # fetch the website url
    url = 'https://zoro.to/most-favorite'
    response = requests.get(url)

    # fetch the anime page
    soup = BeautifulSoup(response.text, 'html.parser')
    page = soup.find('div', class_='tab-content')
    animes = page.find_all('div', class_='flw-item')

    # container to store anime attributes
    mostFavorites = {'title': [], 'genres': [], 'description': [], 'poster': []}

    # start scraping anime
    for i in range(1, 151):
        for anime in animes:
            try:
                # finally start storing the anime attributes
                title, genres, description, poster = fetchAnimeContent(anime)
                mostFavorites['title'].append(title)
                mostFavorites['genres'].append(genres)
                mostFavorites['description'].append(description)
                mostFavorites['poster'].append(poster)
            except:
                continue

        # change the animes page link
        url = f'https://zoro.to/most-favorite?page={i+1}'
        response = requests.get(url)

        # fetch the anime page
        soup = BeautifulSoup(response.text, 'html.parser')
        page = soup.find('div', class_='tab-content')
        animes = page.find_all('div', class_='flw-item')
except Exception as error:
    print('Error Occured:', error)

In [ ]:
most_favorites = pd.DataFrame(mostFavorites)
most_favorites.shape

(5981, 4)

In [ ]:
most_favorites.to_csv('/content/drive/MyDrive/csv_files/most_favorites.csv')